In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210130.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice]
0,1/26/2021 17:36:52,Before the Devil Knows You're Dead (2007),Rocket Singh: Salesman of the Year (2009),Hold the Dark (2018),The Wolf of Snow Hollow (2020),Salesman (1969),OOPS ALL GLENN CLOSE,OOPS! ALL BRENDAN FRASER,MY BAD HABIT IS MY PERSONALITY.,DOWN UNDAH,[Default to Nicolas Cage],A CINEMATIC SHITPOST
1,1/26/2021 17:38:38,Salesman (1969),Before the Devil Knows You're Dead (2007),Rocket Singh: Salesman of the Year (2009),Hold the Dark (2018),The Wolf of Snow Hollow (2020),A CINEMATIC SHITPOST,DOWN UNDAH,MY BAD HABIT IS MY PERSONALITY.,[Default to Nicolas Cage],OOPS ALL GLENN CLOSE,OOPS! ALL BRENDAN FRASER
2,1/26/2021 17:47:02,The Wolf of Snow Hollow (2020),Salesman (1969),Before the Devil Knows You're Dead (2007),Rocket Singh: Salesman of the Year (2009),Hold the Dark (2018),DOWN UNDAH,MY BAD HABIT IS MY PERSONALITY.,OOPS! ALL BRENDAN FRASER,OOPS ALL GLENN CLOSE,A CINEMATIC SHITPOST,[Default to Nicolas Cage]
3,1/26/2021 17:50:20,Rocket Singh: Salesman of the Year (2009),Before the Devil Knows You're Dead (2007),The Wolf of Snow Hollow (2020),Salesman (1969),Hold the Dark (2018),OOPS! ALL BRENDAN FRASER,DOWN UNDAH,MY BAD HABIT IS MY PERSONALITY.,OOPS ALL GLENN CLOSE,[Default to Nicolas Cage],A CINEMATIC SHITPOST
4,1/26/2021 17:52:56,The Wolf of Snow Hollow (2020),Salesman (1969),Rocket Singh: Salesman of the Year (2009),Hold the Dark (2018),Before the Devil Knows You're Dead (2007),MY BAD HABIT IS MY PERSONALITY.,A CINEMATIC SHITPOST,DOWN UNDAH,OOPS! ALL BRENDAN FRASER,OOPS ALL GLENN CLOSE,[Default to Nicolas Cage]
5,1/27/2021 5:56:41,Hold the Dark (2018),The Wolf of Snow Hollow (2020),Before the Devil Knows You're Dead (2007),Salesman (1969),Rocket Singh: Salesman of the Year (2009),OOPS! ALL BRENDAN FRASER,DOWN UNDAH,A CINEMATIC SHITPOST,[Default to Nicolas Cage],NaN,NaN
6,1/27/2021 18:28:41,Before the Devil Knows You're Dead (2007),Hold the Dark (2018),Salesman (1969),The Wolf of Snow Hollow (2020),Rocket Singh: Salesman of the Year (2009),A CINEMATIC SHITPOST,DOWN UNDAH,MY BAD HABIT IS MY PERSONALITY.,OOPS! ALL BRENDAN FRASER,OOPS ALL GLENN CLOSE,NaN


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(["Before the Devil Knows You're Dead (2007)",
       'Rocket Singh: Salesman of the Year (2009)',
       'Hold the Dark (2018)', 'The Wolf of Snow Hollow (2020)',
       'Salesman (1969)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{"Before the Devil Knows You're Dead (2007)": <Candidate('Before the Devil Knows You're Dead (2007)')>,
 'Rocket Singh: Salesman of the Year (2009)': <Candidate('Rocket Singh: Salesman of the Year (2009)')>,
 'Hold the Dark (2018)': <Candidate('Hold the Dark (2018)')>,
 'The Wolf of Snow Hollow (2020)': <Candidate('The Wolf of Snow Hollow (2020)')>,
 'Salesman (1969)': <Candidate('Salesman (1969)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Before the Devil Knows You're Dead (2007), Rocket Singh: Salesman of the Year (2009), Hold the Dark (2018), The Wolf of Snow Hollow (2020), Salesman (1969))>,
 <Ballot(Salesman (1969), Before the Devil Knows You're Dead (2007), Rocket Singh: Salesman of the Year (2009), Hold the Dark (2018), The Wolf of Snow Hollow (2020))>,
 <Ballot(The Wolf of Snow Hollow (2020), Salesman (1969), Before the Devil Knows You're Dead (2007), Rocket Singh: Salesman of the Year (2009), Hold the Dark (2018))>,
 <Ballot(Rocket Singh: Salesman of the Year (2009), Before the Devil Knows You're Dead (2007), The Wolf of Snow Hollow (2020), Salesman (1969), Hold the Dark (2018))>,
 <Ballot(The Wolf of Snow Hollow (2020), Salesman (1969), Rocket Singh: Salesman of the Year (2009), Hold the Dark (2018), Before the Devil Knows You're Dead (2007))>,
 <Ballot(Hold the Dark (2018), The Wolf of Snow Hollow (2020), Before the Devil Knows You're Dead (2007), Salesman (1969), Rocket Singh: Salesman of the Year (2

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                    Votes  Status
-----------------------------------------  -------  --------
Before the Devil Knows You're Dead (2007)        2  Hopeful
The Wolf of Snow Hollow (2020)                   2  Hopeful
Salesman (1969)                                  1  Hopeful
Rocket Singh: Salesman of the Year (2009)        1  Hopeful
Hold the Dark (2018)                             1  Rejected

ROUND 2
Candidate                                    Votes  Status
-----------------------------------------  -------  --------
The Wolf of Snow Hollow (2020)                   3  Hopeful
Before the Devil Knows You're Dead (2007)        2  Hopeful
Salesman (1969)                                  1  Rejected
Rocket Singh: Salesman of the Year (2009)        1  Rejected
Hold the Dark (2018)                             0  Rejected

FINAL RESULT
Candidate                                    Votes  Status
-----------------------------------------  -------  --------
Befor

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                          Votes  Status
-------------------------------  -------  --------
OOPS! ALL BRENDAN FRASER               2  Hopeful
A CINEMATIC SHITPOST                   2  Hopeful
DOWN UNDAH                             1  Hopeful
MY BAD HABIT IS MY PERSONALITY.        1  Hopeful
OOPS ALL GLENN CLOSE                   1  Rejected
[Default to Nicolas Cage]              0  Rejected

ROUND 2
Candidate                          Votes  Status
-------------------------------  -------  --------
OOPS! ALL BRENDAN FRASER               3  Hopeful
A CINEMATIC SHITPOST                   2  Hopeful
DOWN UNDAH                             1  Rejected
MY BAD HABIT IS MY PERSONALITY.        1  Rejected
OOPS ALL GLENN CLOSE                   0  Rejected
[Default to Nicolas Cage]              0  Rejected

FINAL RESULT
Candidate                          Votes  Status
-------------------------------  -------  --------
OOPS! ALL BRENDAN FRASER               4  Elected
A CINEMATIC S